In [24]:
from Bio import Entrez

Entrez.email = "bene6@hotmail.de"  # Always tell NCBI who you are

# Search PubMed for articles with "GC-content" in the title
handle = Entrez.esearch(db="pubmed", term="gc-content[Title]")
record = Entrez.read(handle)

# Get the list of article IDs
article_ids = record["IdList"]

# Fetch the abstracts for the articles
handle = Entrez.efetch(db="pubmed", id=article_ids, rettype="abstract")
records = Entrez.read(handle)

# Print the abstracts
noAbstract = 0
for record in records["PubmedArticle"]:
    try:
        print(record["MedlineCitation"]["Article"]["Abstract"]["AbstractText"][0])
    except KeyError:
        print("No abstract available")
        noAbstract += 1
print("No abstract available for " + str(noAbstract) + " articles"  + "by total of " + str(len(article_ids)) + " articles")


Bacterial evolution is characterized by strong purifying selection as well as rapid adaptive evolution in changing environments. In this context, the genomic GC content (genomic GC) varies greatly but presents some level of phylogenetic stability, making it challenging to explain based on current hypotheses. To illuminate the evolutionary mechanisms of the genomic GC, we analyzed the base composition and functional inventory of 11,083 representative genomes. A phylogenetically constrained bimodal distribution of the genomic GC, which mainly originated from parallel divergences in the early evolution, was demonstrated. Such variation of the genomic GC can be well explained by DNA replication and repair (DRR), in which multiple pathways correlate with the genomic GC. Furthermore, the biased conservation of various stress-related genes, especially the DRR-related ones, implies distinct adaptive processes in the ancestral lineages of high- or low-GC clades which are likely induced by major

In [19]:
records["PubmedArticle"][0]["MedlineCitation"]["Article"]["Abstract"]["AbstractText"][0]

'Bacterial evolution is characterized by strong purifying selection as well as rapid adaptive evolution in changing environments. In this context, the genomic GC content (genomic GC) varies greatly but presents some level of phylogenetic stability, making it challenging to explain based on current hypotheses. To illuminate the evolutionary mechanisms of the genomic GC, we analyzed the base composition and functional inventory of 11,083 representative genomes. A phylogenetically constrained bimodal distribution of the genomic GC, which mainly originated from parallel divergences in the early evolution, was demonstrated. Such variation of the genomic GC can be well explained by DNA replication and repair (DRR), in which multiple pathways correlate with the genomic GC. Furthermore, the biased conservation of various stress-related genes, especially the DRR-related ones, implies distinct adaptive processes in the ancestral lineages of high- or low-GC clades which are likely induced by majo